In [1]:
import os

# Set the proxy environment variables
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [2]:
import ee
import geemap

In [3]:
ee.Initialize()

### Define premise parameter

In [4]:
# Define the time-range
# just get one_range from every different sensors
year_range = [(f'{i}-01-01',f'{i+2}-12-31') for i in range(1990,2023,3)][-1:]

In [5]:
year_range

[('2020-01-01', '2022-12-31')]

##### Adjust below code to your research area

In [10]:
# define region EN/CN dict
region_name_dict = dict(zip(['东北', '华北', '华东',  '西北', '西南', '中南'],
                            ['dongbei', 'huabei', 'huadong', 'xibei', 'xinan', 'zhongnan']))

region_name_dict

{'东北': 'dongbei',
 '华北': 'huabei',
 '华东': 'huadong',
 '西北': 'xibei',
 '西南': 'xinan',
 '中南': 'zhongnan'}

In [11]:
target_regions = ['东北', '华北', '华东',  '中南'] # '西北', '西南'

In [12]:
# define the asset path for exporting
asset_path = 'users/cnbuilt001/nomalized_index'

### [Mean_NDVI, Mean_NDBI, Mean_EVI] calcualtion according to year_range

In [13]:
# loop the target regions
for region_cn in target_regions:

    # get the area shp for export
    Target_area = ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")\
                    .filterMetadata('NAME1','equals',region_cn)
    
    # get the English name of reigon
    region_en = region_name_dict[region_cn]


    # reverse the list so we can export the recent img first
    for span in year_range:
        
        # find the end year to determine the range_name
        start_year = int(span[0].split('-')[0])
        end_year   = int(span[1].split('-')[0])
        range_name = f'{start_year}_{end_year}'
    
        if end_year <= 2010:
            Landsat_img = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA")
            ND_formula = {'NDVI':['B4','B3'],
                              'NDBI':['B5','B4'],
                              'EVI':"2.5 * ((b('B4')-b('B3'))*1.0 / (b('B4')*1.0 + 6.0 * b('B3') - 7.5 * b('B1') + 1.0))"}
    
        elif end_year <= 2013:
            Landsat_img = ee.ImageCollection("LANDSAT/LE07/C01/T1_TOA")
            ND_formula = {'NDVI':['B4','B3'],
                          'NDBI':['B5','B4'],
                          'EVI':"2.5 * ((b('B4')-b('B3'))*1.0 / (b('B4')*1.0 + 6.0 * b('B3') - 7.5 * b('B1') + 1.0))"}
    
        else:
            Landsat_img = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
            ND_formula = {'NDVI':['B5','B4'],
                              'NDBI':['B6','B5'],
                              'EVI':"2.5 * ((b('B5')-b('B4'))*1.0 / (b('B5')*1.0 + 6.0 * b('B4') - 7.5 * b('B2') + 1.0))"}
        
        # compute the normalized difference imgs
        # multiply the NDVI with 100 and change it 
        # into a int-img to reduce space
        NDVI_mean = Landsat_img\
                                .filterBounds(Target_area)\
                                .filterDate(span[0],span[1])\
                                .mean()\
                                .normalizedDifference(ND_formula['NDVI'])\
                                .multiply(100).toInt8()
        
        NDBI_mean = Landsat_img\
                                .filterBounds(Target_area)\
                                .filterDate(span[0],span[1])\
                                .mean()\
                                .normalizedDifference(ND_formula['NDBI'])\
                                .multiply(100).toInt8()
        
        EVI_mean  = Landsat_img\
                                .filterBounds(Target_area)\
                                .filterDate(span[0],span[1])\
                                .mean()\
                                .expression(ND_formula['EVI'])\
                                .multiply(100).toInt8()
    
    
        # ==================== Export =============================
        # define the name              
        name = f'NDVI_NDBI_EVI_{region_en}_{range_name}'
        
        # cast img to ee.Image
        img = ee.Image([NDVI_mean,NDBI_mean,EVI_mean]).rename(['NDVI','NDBI','EVI'])
        
        # export
        task = ee.batch.Export.image.toAsset(   image           = img,
                                                description     = name,
                                                assetId         = f'{asset_path}/{name}',
                                                region          = Target_area.geometry(),
                                                scale           = 30,
                                                maxPixels       = int(1e13)
                                            )
    
        task.start()
    
        # print out the process
        print(f'Exporting Mean-Normalized-img of {name}')


Exporting Mean-Normalized-img of NDVI_NDBI_EVI_dongbei_2020_2022
Exporting Mean-Normalized-img of NDVI_NDBI_EVI_huabei_2020_2022
Exporting Mean-Normalized-img of NDVI_NDBI_EVI_huadong_2020_2022
Exporting Mean-Normalized-img of NDVI_NDBI_EVI_zhongnan_2020_2022


##### Add img to map

In [14]:
one_img = img

In [17]:
# show the map for validation
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.setCenter(120.3482, 33.0212,8)

# add the img
Map.addLayer(one_img,{'min':-10,'max':35},'NDVI_NDBI_EVI')


# show the img
Map